In [3]:
import pandas as pd
import numpy as np
import requests

url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
#Create a handle, page, to handle the contents of the website
page = requests.get(url)
#Store the contents of the website under doc
doc = lh.fromstring(page.content)
#Parse data that are stored between <tr>..</tr> of HTML
tr_elements = doc.xpath('//tr')

# the length of the first 12 rows
[len(T) for T in tr_elements[:12]]

[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]

In [6]:

tr_elements = doc.xpath('//tr')
#Create empty list
col=[]
i=0
#For each row, store each first element (header) and an empty list
for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    print ('%d: %s'%(i,name))
    col.append((name,[]))

1: Postal Code

2: Borough

3: Neighbourhood



## Creating  Panda's Data Frame

In [7]:
#Since our first row is the header, data is stored on the second row onwards
for j in range(1,len(tr_elements)):
    #T is our j'th row
    T=tr_elements[j]
    
    #If row is not of size 3, the //tr data is not from our table 
    if len(T)!=3:
        break
    
    #i is the index of our column
    i=0
    
    #Iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content() 
        #Check if row is empty
        col[i][1].append(data)
        #Increment i for the next column
        i+=1

In [8]:
[len(C) for (title,C) in col]

[181, 181, 181]

In [9]:

Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)

In [10]:
df.head()

,Postal Code\n,Borough\n,Neighbourhood\n
0,M1A\n,Not assigned\n,Not assigned\n
1,M2A\n,Not assigned\n,Not assigned\n
2,M3A\n,North York\n,Parkwoods\n
3,M4A\n,North York\n,Victoria Village\n
4,M5A\n,Downtown Toronto\n,"Regent Park, Harbourfront\n"


In [11]:

df.tail()

,Postal Code\n,Borough\n,Neighbourhood\n
176,M6Z\n,Not assigned\n,Not assigned\n
177,M7Z\n,Not assigned\n,Not assigned\n
178,M8Z\n,Etobicoke\n,"Mimico NW, The Queensway West, South of Bloor,..."
179,M9Z\n,Not assigned\n,Not assigned\n
180,\n,Canadian postal codes\n,\n


# Cleaning the Data 

In [20]:
# strip the "/n" from the values
df_obj = df.select_dtypes(['object'])
df[df_obj.columns] = df_obj.apply(lambda x: x.str.strip('\n'))

In [26]:
df.drop([160], inplace=True)

In [25]:
df.tail()

,Postal Code\n,Borough\n,Neighbourhood\n
175,M5Z,Not assigned,Not assigned
176,M6Z,Not assigned,Not assigned
177,M7Z,Not assigned,Not assigned
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."
179,M9Z,Not assigned,Not assigned


In [27]:

#strip the "/n" from the column names
df.rename(columns=lambda x: x.strip('\n'), inplace=True)

In [28]:
df.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [29]:
#Checking how many Boroughs has Not Assigned values
NABor_df = df.loc[df['Borough'] == "Not assigned"] 
  
print(NABor_df)

    Postal Code       Borough Neighbourhood
0           M1A  Not assigned  Not assigned
1           M2A  Not assigned  Not assigned
7           M8A  Not assigned  Not assigned
10          M2B  Not assigned  Not assigned
15          M7B  Not assigned  Not assigned
..          ...           ...           ...
174         M4Z  Not assigned  Not assigned
175         M5Z  Not assigned  Not assigned
176         M6Z  Not assigned  Not assigned
177         M7Z  Not assigned  Not assigned
179         M9Z  Not assigned  Not assigned

[77 rows x 3 columns]


In [30]:

#delete all rows from the dataset where Borough has a "Not assigned" value
df.drop(df[df.Borough == "Not assigned"].index, inplace=True)

In [31]:

#Check how many Neighbourhoods has Not Assigned values
NANeigh_df = df.loc[df['Neighbourhood'] == "Not assigned"] 
  
print(NANeigh_df)

Empty DataFrame
Columns: [Postal Code, Borough, Neighbourhood]
Index: []


In [32]:
#reseting the index
df.reset_index(drop=True, inplace=True)

In [33]:
df.shape

(102, 3)

# Part 2
## latitude and the longitude coordinates of each neighborhood

In [34]:
geodata = pd.read_csv('https://cocl.us/Geospatial_data')

In [35]:
geodata.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [36]:
geodata.shape

(103, 3)

In [37]:
Tor_geo=df.merge(geodata, how='left', on='Postal Code')

In [38]:
Tor_geo.head(10)

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


# Part 3

## Create a map of Toronto with boroughs superimposed on top
Explore and cluster the neighborhoods in Toronto. You can decide to work with only boroughs that contain the word Toronto and then replicate the same analysis we did to the New York City data. It is up to you.

Just make sure:

to add enough Markdown cells to explain what you decided to do and to report any observations you make. 
Generate maps to visualize your neighborhoods and how they cluster together. Once you are happy with your analysis, submit a link to the new Notebook on your Github repository


In [40]:
# Importing the dependencies to create a map
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python-3.7-main

  added / updated specs:
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    _libgcc_mutex-0.1          |      conda_forge           3 KB  conda-forge
    _openmp_mutex-4.5          |           1_llvm           5 KB  conda-forge
    _py-xgboost-mutex-2.0      |            cpu_0           8 KB  conda-forge
    _pytorch_select-0.2        |            gpu_0           2 KB
    absl-py-0.11.0             |   py37h89c1867_0         168 KB  conda-forge
    aiohttp-3.7.4              |   py37h5e8e339_0         632 KB  conda-forge
    altair-4.1.0               |             py_1         614 KB  conda-forge
   

freetds-1.1.15       | 2.4 MB    | ##################################### | 100% 
numpy-1.20.1         | 5.8 MB    | ##################################### | 100% 
google-pasta-0.2.0   | 42 KB     | ##################################### | 100% 
scipy-1.5.3          | 18.5 MB   | ##################################### | 100% 
scikit-image-0.18.1  | 11.5 MB   | ##################################### | 100% 
jupyterlab_pygments- | 8 KB      | ##################################### | 100% 
json5-0.9.5          | 20 KB     | ##################################### | 100% 
mkl_random-1.2.0     | 313 KB    | ##################################### | 100% 
boto3-1.17.22        | 70 KB     | ##################################### | 100% 
libtiff-4.2.0        | 633 KB    | ##################################### | 100% 
jpeg-9d              | 264 KB    | ##################################### | 100% 
tensorboard-1.14.0   | 3.2 MB    | ##################################### | 100% 
imageio-2.9.0        | 3.1 M

pytables-3.6.1       | 1.5 MB    | ##################################### | 100% 
sqlalchemy-1.3.23    | 1.8 MB    | ##################################### | 100% 
kiwisolver-1.3.1     | 78 KB     | ##################################### | 100% 
hdf5-1.10.6          | 3.1 MB    | ##################################### | 100% 
prometheus_client-0. | 44 KB     | ##################################### | 100% 
keyring-18.0.0       | 50 KB     | ##################################### | 100% 
numexpr-2.7.3        | 201 KB    | ##################################### | 100% 
gstreamer-1.18.3     | 2.0 MB    | ##################################### | 100% 
tifffile-2021.3.5    | 127 KB    | ##################################### | 100% 
sip-4.19.25          | 295 KB    | ##################################### | 100% 
traitlets-5.0.5      | 81 KB     | ##################################### | 100% 
google-auth-1.24.0   | 62 KB     | ##################################### | 100% 
ipykernel-5.5.0      | 166 K

wcwidth-0.2.5        | 33 KB     | ##################################### | 100% 
termcolor-1.1.0      | 6 KB      | ##################################### | 100% 
libsodium-1.0.18     | 366 KB    | ##################################### | 100% 
mysql-common-8.0.23  | 1.5 MB    | ##################################### | 100% 
nspr-4.29            | 232 KB    | ##################################### | 100% 
mock-4.0.3           | 51 KB     | ##################################### | 100% 
pyzmq-22.0.3         | 524 KB    | ##################################### | 100% 
ca-certificates-2020 | 137 KB    | ##################################### | 100% 
libiconv-1.16        | 1.4 MB    | ##################################### | 100% 
libxgboost-1.3.3     | 3.3 MB    | ##################################### | 100% 
ibm-wsrt-py37main-ma | 2 KB      | ##################################### | 100% 
tqdm-4.59.0          | 77 KB     | ##################################### | 100% 
libzopfli-1.0.3      | 164 K

done


In [64]:
!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

Solving environment: done

# All requested packages already installed.



In [65]:
#initiate by getting the coordinates of Toronto center
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="tor_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [66]:
# create map of Toronto using latitude and longitude values
map_Tor = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, Borough, Neighbourhood in zip(Tor_geo['Latitude'], Tor_geo['Longitude'], Tor_geo['Borough'], Tor_geo['Neighbourhood']):
    label = '{}, {}'.format(Neighbourhood, Borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Tor)  
    
map_Tor